In [2]:
import pandas as pd

In [3]:
df1 = pd.read_csv(r'C:/Users/ashuo/DNS/data/dga_classifier_data/normal_domain_names/opendns-top-domains.txt', header=None, names=['domain_name'])
df2 = pd.read_csv(r'C:/Users/ashuo/DNS/data/dga_classifier_data/normal_domain_names/opendns-top-domains.txt', header=None, names=['domain_name'])
df3 = pd.read_csv(r'C:/Users/ashuo/DNS/data/dga_classifier_data/normal_domain_names/non_malicious_domain_names.txt', header=None, names=['domain_name'])
df3 = df3.drop_duplicates()
df_normal = pd.concat([df1, df2, df3], ignore_index=True)

df_normal = df_normal.drop_duplicates()

df_normal['source'] = 'open_dns'
df_normal['label'] = 0

In [9]:
import os
import pandas as pd
import random

def sample_domains_from_files(directory, sample_size):
    all_domains = []
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            with open(os.path.join(directory, filename), 'r') as f:
                domains = f.read().splitlines()
                # Check if the file has enough domains to sample
                if len(domains) >= sample_size:
                    # Randomly sample domains from this file
                    sampled_domains = random.sample(domains, sample_size)
                    # Pair each sampled domain with the filename
                    domains_with_source = [(domain, filename) for domain in sampled_domains]
                    all_domains.extend(domains_with_source)
    
    # Convert to DataFrame
    df = pd.DataFrame(all_domains, columns=['domain_name', 'source'])
    
    return df

# Use the function
directory = r'C:/Users/ashuo/DNS/data/dga_classifier_data/dga_domain_names'  # replace with your actual directory
sample_size = 265
df_dga = sample_domains_from_files(directory, sample_size)

In [10]:
df_dga.shape
df_dga['label'] = 1

In [11]:
df = pd.concat([df_normal, df_dga], ignore_index=True)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [13]:
#  The labeled dataset is split in the ratio of 70:30 where 70 percent is the training data
# and 30 is the test data. The Train and test dataset are then tokenized into a form that is
# acceptable to the transformer model. The CANINE transformer model is then fine-tuned using
# the trained dataset. On achieving the desired accuracy, model fine-tuning is completed.
# The trained model is then evaluated using the test dataset that was set aside for the
# evaluation of the model. The tokenization, CANINE transformer model, 
# and the results of the evaluation are discussed in subsequent sections

In [15]:
from sklearn.model_selection import train_test_split
X = df['domain_name']
y = df['label']

train_texts, test_texts, train_labels, test_labels = train_test_split(X, y, test_size=0.3, random_state=42)

train_texts = train_texts.astype(str).tolist()
test_texts = test_texts.astype(str).tolist()

In [16]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification# Load the CANINE-c tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/canine-c")

# Tokenize training and testing data
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

['somersetdans.ch', 'forniture-alberghiere.biz', 'nopkltypmjrvhxhm.com', 'szboxesco.com', 'csd.it']


In [19]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification# Load the CANINE-c tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/canine-c")

# Tokenize training and testing data
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

C:\Users\ashuo\AppData\Roaming\Python\Python38\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
import torch
from torch.utils.data import Dataset

class DGADataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Convert labels to tensor
train_labels = torch.tensor(train_labels.tolist())
test_labels = torch.tensor(test_labels.tolist())

# Clone and detach tensors
train_labels = train_labels.clone().detach()
test_labels = test_labels.clone().detach()

# Convert encodings and labels into Dataset
train_dataset = DGADataset(train_encodings, train_labels)
test_dataset = DGADataset(test_encodings, test_labels)

In [21]:
from transformers import Trainer, TrainingArguments

# Load CANINE-c model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained("google/canine-c")

training_args = TrainingArguments(
    output_dir='./results',  # output directory
    num_train_epochs=3,  # Total number of training epochs to perform
    per_device_train_batch_size=16,  # Batch size per device during training
    per_device_eval_batch_size=64,  # Batch size for evaluation
    warmup_steps=500,  # Number of steps for the warmup phase
    learning_rate=2e-5,  # Learning rate
    evaluation_strategy="steps",  # Evaluation and save happens at every specified step
    eval_steps=500,  # Number of update steps between two evaluations
    save_steps=500,  # after # steps model is saved 
    logging_steps=500,  # after # steps logs are shown and saved
)




Some weights of CanineForSequenceClassification were not initialized from the model checkpoint at google/canine-c and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
import logging

# Set up logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)

In [23]:
# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Fine-tune the model
trainer.train()

  0%|          | 0/5232 [00:00<?, ?it/s]C:\Users\ashuo\AppData\Local\Temp\ipykernel_1224\2690441442.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])
  1%|          | 29/5232 [00:30<1:36:33,  1.11s/it]

KeyboardInterrupt: 

In [ ]:
# Evaluate the model
print("bvvjkfn")
evaluation_results = trainer.evaluate(test_dataset)

print("helooovnjd")
print(evaluation_results)

bvvjkfn


NameError: name 'trainer' is not defined

In [ ]:
# Get the model's predictions
import numpy as np
predictions = trainer.predict(test_dataset)


# Convert logits to predictions
pred_labels = np.argmax(predictions.predictions, axis=-1)

# Get true labels from the dataset
true_labels = [example['labels'] for example in test_dataset]

# Compute accuracy
accuracy = (pred_labels == true_labels).mean()

print(f'Accuracy: {accuracy}')

C:\Users\ashuo\AppData\Local\Temp\ipykernel_17576\2690441442.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])
100%|██████████| 5/5 [00:02<00:00,  1.98it/s]

Accuracy: 0.6966666666666667
